In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
tweet= pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
tweet.head(3)

def create_corpus(target):
    corpus=[]
    
    for x in tweet[tweet['target']==target]['text'].str.split():
        for i in x:
            corpus.append(i)
    return corpus

corpus=create_corpus(0)
dic=defaultdict(int)
for word in corpus:
    if word in stop:
        dic[word]+=1
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:10]
df=pd.concat([tweet,test])
example="New competition launched :https://www.kaggle.com/c/nlp-getting-started"
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

remove_URL(example)
df['text']=df['text'].apply(lambda x : remove_html(x))

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

df['text']=df['text'].apply(lambda x : remove_html(x))

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df['text']=df['text'].apply(lambda x: remove_emoji(x))

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

example="I am a #king"
print(remove_punct(example))
df['text']=df['text'].apply(lambda x : remove_punct(x))
!pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
text = "corect me plese"
correct_spellings(text)
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

corpus=create_corpus(df)

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove\.\6B.zip



I am a king
     |████████████████████████████████| 1.9MB 3.4MB/s 


100%|██████████| 10876/10876 [00:01<00:00, 6270.85it/s]


--2020-04-30 12:00:40--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-04-30 12:00:40--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-04-30 12:00:41--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
embedding_dict={}
with open('glove.6B.100d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()
MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')
word_index=tokenizer_obj.word_index
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

train=tweet_pad[:tweet.shape[0]]
test=tweet_pad[tweet.shape[0]:]
X_train,X_test,y_train,y_test=train_test_split(train,tweet['target'].values,test_size=0.15)

sample_sub=pd.read_csv('sample_submission.csv')



bilstm=Sequential()

embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

bilstm.add(embedding)
bilstm.add(SpatialDropout1D(0.2))
bilstm.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
bilstm.add(Dense(64, activation='relu'))
bilstm.add(Dense(1, activation='sigmoid'))
optimzer=Adam()

bilstm.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

optimzer=Adam(lr=1e-4)

bilstm.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])
history=bilstm.fit(X_train,y_train,batch_size=64,epochs=15,validation_data=(X_test,y_test),verbose=1)

100%|██████████| 21444/21444 [00:00<00:00, 610919.71it/s]


Train on 6471 samples, validate on 1142 samples
Epoch 1/15
6471/6471 [==============================] - 24s 4ms/step - loss: 0.6728 - accuracy: 0.5866 - val_loss: 0.6250 - val_accuracy: 0.6935
Epoch 2/15
6471/6471 [==============================] - 22s 3ms/step - loss: 0.5693 - accuracy: 0.7291 - val_loss: 0.4949 - val_accuracy: 0.7706
Epoch 3/15
6471/6471 [==============================] - 21s 3ms/step - loss: 0.5223 - accuracy: 0.7594 - val_loss: 0.4757 - val_accuracy: 0.7811
Epoch 4/15
6471/6471 [==============================] - 22s 3ms/step - loss: 0.5044 - accuracy: 0.7646 - val_loss: 0.4659 - val_accuracy: 0.7881
Epoch 5/15
6471/6471 [==============================] - 22s 3ms/step - loss: 0.4894 - accuracy: 0.7750 - val_loss: 0.4570 - val_accuracy: 0.7960
Epoch 6/15
6471/6471 [==============================] - 22s 3ms/step - loss: 0.4876 - accuracy: 0.7741 - val_loss: 0.4510 - val_accuracy: 0.7960
Epoch 7/15
6471/6471 [==============================] - 21s 3ms/step - loss: 0.476

In [0]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 3.4MB/s 


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import tokenization

In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)
trainb = pd.read_csv("train.csv")
testb = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
train_input = bert_encode(trainb.text.values, tokenizer, max_len=160)
test_input = bert_encode(testb.text.values, tokenizer, max_len=160)
train_labels = trainb.target.values
bert = build_model(bert_layer, max_len=160)
bert.summary()
train_history = bert.fit(
    train_input, train_labels,
    validation_split=0.1,
    epochs=2,
    batch_size=12
)

bert.save('bert.h5')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [0]:


from sklearn.linear_model import LinearRegression, LogisticRegression
import numpy as np
class Ensemble_models:
  def __init__(self,n_models):
    self.n_models = n_models
    self.regressor = LogisticRegression()  
  

  def __extract_features__(self,predictions):
    features = [predictions[i] for i in range(self.n_models)]
    for i in range(self.n_models):
      for j in range(self.n_models):
        if i==j:
          continue
        features.append(predictions[i]*predictions[j])

    return np.array(features).T.reshape(len(features[1]),-1)

  def fit(self,predictions,ground_true):
    features =self.__extract_features__(predictions)
    print(features,ground_true)
    self.regressor.fit(features, ground_true)

  def predict(self,predictions):
    features =self.__extract_features__(predictions)
    print(self.regressor.coef_)
    return self.regressor.predict(features)



In [0]:
def predict_kaggle(predictions):
  y_pre=predictions
  
  y_pre=np.round(y_pre).astype(int).reshape(3263)
  sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':y_pre})
  sub.to_csv('submission.csv',index=False)

In [0]:
models = [bilstm,bert]
bert_pred=bert.predict(train_input)
bilst_pred =bilstm.predict(X_train)
predictions = [bilst_pred,bert_pred[:len(bilst_pred)]]
en_models = Ensemble_models(len(models))
en_models.fit(predictions,y_train)
answers = [bilstm.predict(test),bert.predict(test_input)]
answer = en_models.predict(answers)
predict_kaggle(answer)

[[0.14231974 0.97749925 0.13911743 0.13911743]
 [0.14903761 0.9830027  0.14650439 0.14650439]
 [0.10162592 0.95558417 0.09711212 0.09711212]
 ...
 [0.14860359 0.2894278  0.04301001 0.04301001]
 [0.9698935  0.10754597 0.10430814 0.10430814]
 [0.08722026 0.03001508 0.00261792 0.00261792]] [0 0 0 ... 0 1 0]
[[5.31509387 0.10066894 0.0676214  0.0676214 ]]
